# 연관규칙 탐색

In [20]:
import os
os.chdir(r"D:\패스트캠퍼스\데이터전처리\part-2.-탐색적-데이터-분석\2. 탐색적 데이터 분석\데이터")

import pandas as pd
df = pd.read_csv("instacart Market Basket Analysis.csv")

In [21]:
df.head()

,order_id,product_id,add_to_cart_order,reordered
0,1,49302,1,1
1,1,11109,2,1
2,1,10246,3,0
3,1,49683,4,0
4,1,43633,5,1


In [22]:
df['product_id'].value_counts(normalize = True).head(100)

24852    0.013524
13176    0.011180
21137    0.007868
21903    0.007066
47626    0.005875
           ...   
8193     0.001024
9387     0.000996
37687    0.000984
20995    0.000983
34243    0.000976
Name: product_id, Length: 100, dtype: float64

In [23]:
# 아이디별로 구매한 아이템들을 list화
product_list_per_order = df.groupby('order_id')['product_id'].apply(list)

In [33]:
# 구매 기록 데이터 -> one hot encoding데이터
from mlxtend.preprocessing import TransactionEncoder
encoder = TransactionEncoder()
one_hot_df = encoder.fit(product_list_per_order).transform(product_list_per_order) # 결과 : ndarray
one_hot_df = pd.DataFrame(one_hot_df, columns=encoder.columns_)
one_hot_df.head()

MemoryError: Unable to allocate 4.78 GiB for an array with shape (131209, 39123) and data type bool

In [25]:
# 연관규칙 탐색
from mlxtend.frequent_patterns import *
frequent_item_df = apriori(one_hot_df, min_support=0.003) # 0.3% 이상 구매한 상품만 대상으로 함
result = association_rules(frequent_item_df, metric = 'confidence', min_threshold = 0.1)
result[['antecedents', 'consequents', 'support', 'confidence']].sort_values(by='confidence', ascending=False).to_csv("연관규칙탐사결과.csv", index=False)

MemoryError: Unable to allocate 31.1 GiB for an array with shape (127260, 2, 131209) and data type bool

메모리문제로,, 중단 결과는 확인하지 않도록 한다..

In [31]:
# 메모리 복구를 위해 변수 삭제
import gc

del one_hot_df
gc.collect()

NameError: name 'one_hot_df' is not defined

# 빈발 시퀀스 탐색

In [5]:
import os
import numpy as np
os.chdir(r"D:\패스트캠퍼스\데이터전처리\part-2.-탐색적-데이터-분석\2. 탐색적 데이터 분석\데이터")

import pandas as pd
df = pd.read_csv("페이지내_사용자_이동.csv", engine="python")

In [6]:
df.head()

,고객ID,방문 페이지,순서
0,0,페이지C,1
1,0,페이지E,2
2,0,페이지B,3
3,0,페이지F,4
4,0,페이지C,5


In [7]:
# 주문별 카트에 추가한 순서를 고려하기 위해, 정렬이 필요
df.sort_values(by = ['고객ID', '순서'], inplace = True)

In [8]:
page_set = df['방문 페이지'].unique()
page_set[:10]

array(['페이지C', '페이지E', '페이지B', '페이지F', '페이지D', '페이지J', '메인', '페이지G',
       '페이지A', '페이지I'], dtype=object)

In [9]:
page_sequence_per_order = df.groupby('고객ID')['방문 페이지'].apply(np.array)

In [10]:
page_sequence_per_order.head()

고객ID
0    [페이지C, 페이지E, 페이지B, 페이지F, 페이지C, 페이지D, 페이지J, 메인,...
1    [페이지B, 메인, 메인, 페이지A, 페이지F, 페이지C, 페이지I, 페이지E, 페...
2    [페이지F, 페이지J, 페이지D, 페이지G, 페이지C, 메인, 페이지I, 페이지J,...
3    [페이지I, 페이지I, 페이지J, 페이지J, 페이지C, 페이지A, 페이지H, 페이지...
4    [페이지J, 페이지H, 페이지G, 페이지G, 페이지E, 페이지C, 페이지A, 페이지...
Name: 방문 페이지, dtype: object

In [26]:
from itertools import product # 둘 이상의 아이템 집합의 가능한 모든 조합 나타냄

# 특정 레코드 하에 특정 패턴이 발생했는지
# L은 윈도우의 크기
def contain_pattern(record, pattern, L):
    output = False # 등장X로 초기화
    if set(record) & set(pattern) != set(pattern): # pattern에 포함된 모든 아이템 집합이 record에 포함된 아이템 집합에 속하지 않을때
        return False
    else:
        # 패턴에 속한 개별 아이템에 대한 위치를 미리 구하기
        pattern_index_list = [np.where(record == item)[0] for item in pattern]
        
    '가능한 모든 조합에서 위치 간 거리가 L이하면 True를 반환' 
    'record = [A, B, C, A, C, C], pattern = [A, B], L = 1'
    'A의 위치: [0, 3], B의 위치: [1]'
    '가능한 모든 조합: [0, 1], [3, 1]'
    '가능한 모든 조합의 거리 차이: [1 - 0, 1 - 3] 중에 0번째 요소는 만족하므로 True'
    
    for pattern_index in product(*pattern_index_list): # * : 각각의 값에 있는 모든 조합을 찾는다
        distance = np.array(pattern_index)[1:] - np.array(pattern_index)[:-1]
        if sum((distance <= L) & (distance > 0)) == (len(pattern_index)-1): # 경우의 수가 모든 패턴에 대해 일치
            output = True
            break
            
    return output

In [27]:
def find_maximum_frequent_sequence_item(item_set, sequence_data, min_support=0.01, L=1):
    queue=[]
    maximum_frequent_sequence_item = []
    
    # 유니크한 아이템 집합에 대해, min_support가 넘는 아이템들만 queue에 추가시킴
    for item in item_set:
        occurence = sequence_data.apply(contain_pattern, pattern = [item], L=L).sum()
        if occurence / len(sequence_data) >= min_support:
            queue.append([item])
            
    while queue:
        current_pattern = queue.pop() # 맨 마지막 값 빼기
        check_maximum_frequent = True # 모든 자식 집합이 min_support를 넘기지 않으면 True를 유지
        for item in item_set :
            occurence = sequence_data.apply(contain_pattern, pattern = current_pattern + [item], L=L).sum()
            if occurence / len(sequence_data) >= min_support: #  min_support를 넘는 패턴을 queue에 추가
                check_maximum_frequent = False
                queue.append(current_pattern + [item])
        
        if check_maximum_frequent and len(current_pattern)>1:
            maximum_frequent_sequence_item.append(current_pattern)
            
    
    return maximum_frequent_sequence_item

In [28]:
def generate_association_rules(maximum_frequent_sequence_item, sequence_data, min_support = 0.01, min_confidence=0.5, L=1):
    # 결과 초기화
    result = {"부모":[], "자식":[], "지지도":[], "신뢰도":[]}
    
    for sequence_item in maximum_frequent_sequence_item:
        # A->B에서 A, B를 모두 포함하는 가짓 수 co_occurence 계산
        co_occurence = sequence_data.apply(contain_pattern, pattern = sequence_item, L=L).sum()
        support = co_occurence / len(sequence_data)
        if co_occurence > min_support:
            for i in range(len(sequence_item)-1, 0, -1): # 한 아이템 집합에 대해 부모 크기를 1씩 줄여나가는 방식
                antecedent = sequence_item[:i]
                consequent = sequence_item[i:]
                antecedent_occurence = sequence_data.apply(contain_pattern, pattern = antecedent, L=L).sum()
                
                confidence = co_occurence / antecedent_occurence
                if confidence>min_confidence:
                    result['부모'].append(antecedent)
                    result['자식'].append(consequent)
                    result['지지도'].append(support)
                    result['신뢰도'].append(confidence)
                
    return pd.DataFrame(result)
                    

In [29]:
maximum_frequent_sequence_item = find_maximum_frequent_sequence_item(page_set, page_sequence_per_order, min_support = 0.02, L=1)

In [30]:
result = generate_association_rules(maximum_frequent_sequence_item, page_sequence_per_order, min_support=0.01, min_confidence=0, L=1)
result.sort_values(by = ['지지도', '신뢰도'], ascending = False)

,부모,자식,지지도,신뢰도
160,[페이지B],[페이지J],0.07,0.155556
59,[페이지A],[페이지C],0.07,0.148936
109,[페이지J],[페이지D],0.07,0.142857
115,[페이지J],[페이지C],0.07,0.142857
75,[메인],[페이지H],0.06,0.150000
...,...,...,...,...
205,[페이지C],"[메인, 페이지A, 페이지J]",0.02,0.037037
207,[페이지C],"[메인, 페이지E]",0.02,0.037037
209,[페이지C],"[페이지J, 페이지I]",0.02,0.037037
214,[페이지C],"[페이지E, 메인]",0.02,0.037037
